In [24]:
import math
import torch
from torch.utils.data import TensorDataset, random_split
from matplotlib import pyplot as plt
import numpy as np 
import emcee
import time 
from tqdm import tqdm
import corner
import h5py

import gpytorch
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel
from gpytorch.models import ExactGP
from gpytorch.likelihoods import MultitaskGaussianLikelihood
from gpytorch.distributions import MultitaskMultivariateNormal


# Import data 

In [25]:
def load_data_as_tensors(filename):
    keys = []
    values = []
    with h5py.File(filename, 'r') as hf:
        for group_name in hf.keys():
            key = hf[group_name]['input_cosmo'][:]
            value = hf[group_name]['LogdL'][:]
            keys.append(key)
            values.append(value)

    keys = np.array(keys)
    values = np.array(values)

    # Convert arrays to PyTorch tensors
    data_x = torch.tensor(keys, dtype=torch.float32)  # Assuming keys are numeric
    data_y = torch.tensor(values, dtype=torch.float32)
    
    # Create a dataset from the tensors
    dataset = TensorDataset(data_x, data_y)
    
    # Determine the split sizes
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    
    # Split the dataset into training and testing sets
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    # Unpack the datasets into tensors
    train_x = torch.stack([data[0] for data in train_dataset])
    train_y = torch.stack([data[1] for data in train_dataset])
    test_x = torch.stack([data[0] for data in test_dataset])
    test_y = torch.stack([data[1] for data in test_dataset])
    
    return train_x, train_y, test_x, test_y

# Load the data and split it
train_x, train_y, test_x, test_y = load_data_as_tensors('LogdL_trial.h5')


In [26]:
train_y.shape

torch.Size([3300, 500])

In [4]:
# class MultitaskGPModel(ExactGP):
#     def __init__(self, train_x, train_y, likelihood):
#         super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
#         self.mean_module = ConstantMean()
#         self.covar_module = ScaleKernel(RBFKernel(ard_num_dims=train_x.size(-1)))

#     def forward(self, x):
#         mean_x = self.mean_module(x)
#         covar_x = self.covar_module(x)
#         return MultitaskMultivariateNormal.from_batch_mvn(
#             gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
#         )

class MultitaskGPModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(RBFKernel(ard_num_dims=train_x.size(-1)))

    def forward(self, x):
        mean_x = self.mean_module(x).unsqueeze(-1).expand(-1, self.likelihood.num_tasks)
        covar_x = self.covar_module(x)
        task_covar = gpytorch.kernels.IndexKernel(num_tasks=self.likelihood.num_tasks, rank=1)
        covar_x = covar_x.add_jitter(1e-4).evaluate()
        task_covar = task_covar(torch.arange(self.likelihood.num_tasks)).evaluate()
        covar_x = gpytorch.lazy.KroneckerProductLazyTensor(covar_x, task_covar)
        
        return MultitaskMultivariateNormal(mean_x, covar_x)


In [6]:
likelihood = MultitaskGaussianLikelihood(num_tasks=train_y.size(-1))   # Number of z values, which is 500
model = MultitaskGPModel(train_x, train_y, likelihood)


In [6]:
model.train()
likelihood.train()

# Use the Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

num_iterations = 50
for i in range(num_iterations):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()
    print(f'Iteration {i + 1}/{num_iterations} - Loss: {loss.item()}')


Iteration 1/50 - Loss: 1.0838841199874878
Iteration 2/50 - Loss: 1.0473910570144653
Iteration 3/50 - Loss: 1.0101312398910522
Iteration 4/50 - Loss: 0.9720796346664429
Iteration 5/50 - Loss: 0.9332265257835388
Iteration 6/50 - Loss: 0.8936579823493958
Iteration 7/50 - Loss: 0.8532719612121582
Iteration 8/50 - Loss: 0.8121502995491028
Iteration 9/50 - Loss: 0.770314633846283
Iteration 10/50 - Loss: 0.7277572751045227
Iteration 11/50 - Loss: 0.684590756893158
Iteration 12/50 - Loss: 0.6406735777854919
Iteration 13/50 - Loss: 0.5961640477180481
Iteration 14/50 - Loss: 0.5510638356208801
Iteration 15/50 - Loss: 0.5053346753120422
Iteration 16/50 - Loss: 0.4590665400028229
Iteration 17/50 - Loss: 0.41231849789619446
Iteration 18/50 - Loss: 0.36502665281295776
Iteration 19/50 - Loss: 0.3172742426395416
Iteration 20/50 - Loss: 0.2690575122833252
Iteration 21/50 - Loss: 0.22048652172088623
Iteration 22/50 - Loss: 0.17146076261997223
Iteration 23/50 - Loss: 0.1221318170428276
Iteration 24/50 - 

In [12]:
## Save the model 
torch.save(model.state_dict(), 'model_state.pth')

In [8]:
# ## When I execute this, it overflows my memory. The dense covariance matrix that I calculated in the training step 
# model.eval()
# likelihood.eval()

# with torch.no_grad(), gpytorch.settings.fast_pred_var():
#     observed_pred = likelihood(model(test_x))
    
# # The output is a MultitaskMultivariateNormal. To get the mean predictions:
# mean_predictions = observed_pred.mean

# # mean_predictions will be of shape (num_samples, 500)


In [7]:
state_dict = torch.load('model_state.pth')
model = MultitaskGPModel(train_x, train_y, likelihood)  # Create a new GP model

model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
model.state_dict()

OrderedDict([('likelihood.raw_task_noises',
              tensor([-5.1974, -5.1967, -5.1973, -5.1970, -5.1976, -5.1972, -5.1971, -5.1974,
                      -5.1971, -5.1975, -5.1979, -5.1971, -5.1972, -5.1975, -5.1976, -5.1967,
                      -5.1975, -5.1971, -5.1972, -5.1973, -5.1967, -5.1975, -5.1973, -5.1966,
                      -5.1966, -5.1968, -5.1976, -5.1973, -5.1972, -5.1970, -5.1976, -5.1972,
                      -5.1974, -5.1975, -5.1974, -5.1975, -5.1978, -5.1972, -5.1980, -5.1971,
                      -5.1974, -5.1975, -5.1978, -5.1977, -5.1980, -5.1976, -5.1978, -5.1974,
                      -5.1973, -5.1971, -5.1965, -5.1969, -5.1973, -5.1972, -5.1973, -5.1975,
                      -5.1976, -5.1971, -5.1974, -5.1974, -5.1974, -5.1972, -5.1971, -5.1972,
                      -5.1972, -5.1974, -5.1973, -5.1965, -5.1975, -5.1974, -5.1979, -5.1981,
                      -5.1975, -5.1973, -5.1969, -5.1980, -5.1981, -5.1970, -5.1974, -5.1970,
                

In [23]:
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # Generate predictions in batches to avoid large memory usage
    batch_size = 1  # Adjust this based on your available memory
    num_batches = int(np.ceil(len(test_x) / batch_size))
    
    mean_predictions = []
    
    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(test_x))
        test_batch = test_x[batch_start:batch_end]
        
        # Make predictions for the current batch
        observed_pred = model(test_batch)
        mean_predictions_batch = likelihood(observed_pred).mean
        
        mean_predictions.append(mean_predictions_batch)
    
    # Concatenate all batch predictions
    mean_predictions = torch.cat(mean_predictions, dim=0)

# mean_predictions will be of shape (num_samples, 500)
print(mean_predictions.shape)


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 10890000000000 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(5, 4))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])